In [5]:
!pip install gradio -q


In [10]:
# ================================================================
# 🚀 Shopping Cart Billing — FINAL RELEASE + Boundary 5/5 Suite
# Topic: Boundary, Confidence & Equivalence Class Partitioning (ECP)
# Student: Muhammad Abdullah | Roll: F22BSEEN1M01131 | Semester: 7th (4M)
# Runs perfectly in Google Colab
# ================================================================

import gradio as gr
import pandas as pd
from datetime import datetime
import uuid, textwrap, os

# --------------------- Core Logic ---------------------
def card_error(msg):
    return f"""
    <div class="card error shake">
      <div class="card-title">⚠️ Input Error</div>
      <p>{msg}</p>
      <p style="margin-top:6px;opacity:.8">Try sample buttons below (Boundary 500 / 501 or Boundary Suite 5/5).</p>
    </div>
    """

def make_receipt_html(ts, amt, cust, pay, disc, final_amt, reasons):
    file_id = uuid.uuid4().hex[:8]
    path = f"/tmp/receipt_{file_id}.html"
    html = f"""<!doctype html>
<html>
<head><meta charset="utf-8"><title>AI-SmartCart Receipt</title>
<style>
  body {{ font-family: Inter, Arial, sans-serif; background:#f5f7fb; margin:0; padding:24px; }}
  .wrap {{ max-width:600px; margin:0 auto; }}
  .head {{ background:linear-gradient(135deg,#0d47a1,#2196f3); color:#fff; padding:16px 18px; border-radius:12px 12px 0 0; }}
  .brand {{ font-weight:800; font-size:20px }}
  .sub {{ opacity:.95; font-size:13px }}
  .card {{ background:#fff; border:1px solid rgba(13,71,161,.15); border-top:none; padding:18px; border-radius:0 0 12px 12px; }}
  .row {{ display:flex; justify-content:space-between; margin:4px 0; }}
  .total {{ background:#eefaf0; border:1px dashed #9ccc65; padding:10px; border-radius:10px; }}
  .badges span {{ display:inline-block; margin:4px 6px 0 0; padding:6px 10px; font-size:12px; border-radius:999px; background:#e3f2fd; border:1px solid rgba(13,71,161,.2) }}
  .foot {{ text-align:center; color:#607d8b; font-size:12px; margin-top:12px }}
  .line {{ border:0; border-top:1px dashed #cfd8dc; margin:12px 0 }}
  @media print {{ body{{ background:#fff }} .wrap{{ box-shadow:none }} }}
</style></head>
<body>
<div class="wrap">
  <div class="head"><div class="brand">AI-SmartCart™ — Official Receipt</div><div class="sub">Generated at {ts}</div></div>
  <div class="card">
    <div class="row"><span>Original Amount</span><b>Rs {amt:.2f}</b></div>
    <div class="row"><span>Customer Type</span><b>{cust}</b></div>
    <div class="row"><span>Payment Type</span><b>{pay}</b></div>
    <div class="row"><span>Discount Applied</span><b>{disc}%</b></div>
    <hr class="line">
    <div class="total row"><span>Final Bill Amount</span><b>Rs {final_amt:.2f}</b></div>
    <hr class="line"><div><b>Discount Details:</b></div>
    <div class="badges">{''.join([f'<span>{r}</span>' for r in reasons])}</div>
    <div class="foot">Thank you for shopping with <b>AI-SmartCart™</b><br/>Print this page (Ctrl/Cmd + P) to save as PDF.</div>
  </div>
</div>
</body></html>"""
    with open(path, "w", encoding="utf-8") as f:
        f.write(html)
    return path

def calc_once(amount, customer_type, payment_type):
    """Return dict with all fields for single case."""
    amt = float(amount)
    discount = 0
    reasons = []
    if payment_type.lower() == "hbl":
        discount += 5; reasons.append("🏦 HBL Payment (+5%)")
    if customer_type.lower() == "member" and amt > 500:
        discount += 10; reasons.append("🧍 Member & Bill > 500 (+10%)")
    elif customer_type.lower() == "guest" and amt > 500:
        discount += 5; reasons.append("👤 Guest & Bill > 500 (+5%)")
    if not reasons:
        reasons.append("ℹ️ No Discount (Bill ≤ 500)")
    final_amt = round(amt * (1 - discount/100), 2)
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return {
        "ts": ts,
        "amt": amt,
        "cust": customer_type.title(),
        "pay": payment_type.upper(),
        "disc": discount,
        "final": final_amt,
        "reasons": reasons
    }

def compute_bill(amount, customer_type, payment_type):
    # validation
    try:
        amt = float(amount)
    except:
        return card_error("Invalid amount! Please enter a number > 0."), "", pd.DataFrame(), None
    if amt <= 0:
        return card_error("Invalid amount! Bill must be greater than 0."), "", pd.DataFrame(), None

    r = calc_once(amt, customer_type, payment_type)

    html = f"""
    <div class="card result slide-up">
      <div class="card-title">🧾 Final Bill Summary</div>
      <div class="kv"><span>Original Amount</span><b>Rs {r['amt']:.2f}</b></div>
      <div class="kv"><span>Customer Type</span><b>{r['cust']}</b></div>
      <div class="kv"><span>Payment Type</span><b>{r['pay']}</b></div>
      <div class="kv"><span>Discount Applied</span><b>{r['disc']}%</b></div>
      <div class="kv green"><span>Final Bill Amount</span><b>Rs {r['final']:.2f}</b></div>
      <div class="stamp">Generated at {r['ts']}</div>
      <hr/><div class="card-sub">📋 Discount Details</div>
      <div class="badges">{''.join([f'<span class="badge">{x}</span>' for x in r['reasons']])}</div>
      <div class="footer">Thank you for shopping with <b>AI-SmartCart™</b></div>
    </div>
    """

    details = textwrap.dedent(f"""
    **Decision Table (applied)**
    - HBL account → +5% (priority)
    - If Bill > 500: Member → +10%, Guest → +5%
    - Otherwise → No discount

    **Why this result?**
    {'  \n'.join(['• ' + x for x in r['reasons']])}
    """)

    row = pd.DataFrame([{
        "Time": r["ts"].split(" ")[1],
        "Amount": r["amt"],
        "Customer": r["cust"],
        "Payment": r["pay"],
        "Discount %": r["disc"],
        "Final (Rs)": r["final"]
    }])

    receipt_path = make_receipt_html(r["ts"], r["amt"], r["cust"], r["pay"], r["disc"], r["final"], r["reasons"])
    return html, details, row, receipt_path

# -------- Boundary 5/5 Suite --------
BOUNDARY_AMOUNTS = [1, 2, 500, 501, 10000]  # min, min+1, on-threshold, just-above, max

def boundary_suite(cust, pay, hist_df):
    """Run 5/5 boundary cases using current customer/payment."""
    cards = []
    rows = []
    receipt_path = None  # only for single run; suite doesn't overwrite file

    for a in BOUNDARY_AMOUNTS:
        r = calc_once(a, cust, pay)
        card = f"""
        <div class="card result mini slide-up">
          <div class="card-title">Case: Rs {r['amt']:.2f}</div>
          <div class="kv"><span>Customer</span><b>{r['cust']}</b></div>
          <div class="kv"><span>Payment</span><b>{r['pay']}</b></div>
          <div class="kv"><span>Discount %</span><b>{r['disc']}</b></div>
          <div class="kv green"><span>Final</span><b>Rs {r['final']:.2f}</b></div>
          <div class="badges">{''.join([f'<span class="badge">{x}</span>' for x in r['reasons']])}</div>
        </div>"""
        cards.append(card)
        rows.append({
            "Time": r["ts"].split(" ")[1],
            "Amount": r["amt"],
            "Customer": r["cust"],
            "Payment": r["pay"],
            "Discount %": r["disc"],
            "Final (Rs)": r["final"]
        })

    html = "<div style='display:grid;grid-template-columns:repeat(auto-fit,minmax(230px,1fr));gap:12px;'>" + "".join(cards) + "</div>"

    # Explanation block
    details = textwrap.dedent("""
    **Boundary Suite (5/5) Used**
    - **min** = 1
    - **min + 1** = 2
    - **on threshold** = 500
    - **just above** = 501
    - **max** = 10000

    This covers classical **5-point BVA** for the Amount domain (1..10000) and the decision boundary at 500.
    """)

    new_rows = pd.DataFrame(rows)
    if isinstance(hist_df, pd.DataFrame) and not hist_df.empty:
        out_df = pd.concat([hist_df, new_rows], ignore_index=True)
    else:
        out_df = new_rows
    return html, details, out_df, out_df, receipt_path

# --------------------- UI ---------------------
with gr.Blocks(css=r"""
:root{ --primary:#0d47a1; --accent:#ff7a18; }
*{font-family: Inter, ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto}
.gradio-container{max-width:1100px !important}
.header{
  background: linear-gradient(135deg,#0d47a1 0%, #2196f3 50%, #64b5f6 100%);
  color:#fff; padding:18px; border-radius:14px; box-shadow:0 10px 24px rgba(13,71,161,.25);
  text-align:center;
}
.header .title{font-size:26px; font-weight:800; letter-spacing:.2px}
.header .meta{opacity:.95; margin-top:6px}
.card{
  background: linear-gradient(180deg, rgba(255,255,255,.95), rgba(255,255,255,.88));
  backdrop-filter: blur(8px);
  border:1.5px solid rgba(13,71,161,.15);
  border-radius:14px; padding:16px; box-shadow:0 10px 24px rgba(13,71,161,.06);
}
.card .card-title{font-weight:800; font-size:18px; margin-bottom:8px; color:var(--primary)}
.card .card-sub{font-weight:700; margin:.3rem 0 .4rem}
.card.result{border-color: rgba(76,175,80,.35)}
.card.result.mini .card-title{font-size:16px}
.card.error{border-color:#ef5350; background:linear-gradient(180deg,#fff5f5,#ffecec)}
.kv{display:flex; justify-content:space-between; padding:6px 8px; border-radius:10px; margin:3px 0; background:#f6f9ff}
.kv.green{background:#eefaf0; border:1px dashed #9ccc65}
.badges .badge{
  display:inline-block; margin:4px 6px 0 0; padding:6px 10px; font-size:12px; border-radius:999px;
  background:#e3f2fd; border:1px solid rgba(13,71,161,.2)
}
hr{border:none;border-top:1px dashed rgba(0,0,0,.12);margin:10px 0}
.btn-row .gr-button{height:44px; font-weight:700}
.btn-accent{background:linear-gradient(90deg,#ff7a18,#ffb24a) !important; color:#111 !important; border:none}
.btn-ghost{background:#f4f6fb !important; color:#0d47a1 !important; border:1px solid rgba(13,71,161,.2)!important}
.stamp{margin-top:8px; font-size:12px; color:#607d8b}
.slide-up{animation:su .35s ease-out both}
.shake{animation:shake .25s linear both}
@keyframes su{from{opacity:0; transform:translateY(10px)} to{opacity:1; transform:none}}
@keyframes shake{
  10%,90%{transform:translateX(-1px)} 20%,80%{transform:translateX(2px)}
  30%,50%,70%{transform:translateX(-4px)} 40%,60%{transform:translateX(4px)}
}
""") as app:

    gr.HTML("""
    <div class='header'>
      <div class='title'>🎓 The Islamia University of Bahawalpur — Software Testing</div>
      <div class='meta'>
        <b>Topic:</b> Boundary, Confidence & Equivalence Partitioning (ECP) &nbsp;|&nbsp;
        <b>Student:</b> Muhammad Abdullah &nbsp;|&nbsp;
        <b>Roll No:</b> F22BSEEN1M01131 &nbsp;|&nbsp;
        <b>Semester:</b> 7th (4M)
      </div>
    </div>
    """)

    gr.Markdown("### 🎨 Shopping Cart Billing — Final Release (with Boundary 5/5)\nThis app applies **Boundary Value Analysis**, **Confidence Testing**, and **Equivalence Class Partitioning** to compute discounts and generate receipts.")

    history_state = gr.State(pd.DataFrame())

    with gr.Row():
        with gr.Column(scale=6):
            with gr.Group():
                gr.HTML("<div class='card'><div class='card-title'>🧩 Inputs</div>")
                amount = gr.Textbox(label="💵 Bill Amount (Rs)", value="600", placeholder="e.g., 750")
                cust   = gr.Radio(["Guest","Member"], label="🧍 Customer Type", value="Guest")
                pay    = gr.Radio(["HBL","Other"], label="🏦 Payment Type", value="Other")
                gr.HTML("</div>")

            with gr.Row(elem_classes="btn-row"):
                submit_btn = gr.Button("Submit", elem_classes="btn-accent")
                clear_btn  = gr.Button("Clear",  elem_classes="btn-ghost")
                b500_btn   = gr.Button("Boundary 500", elem_classes="btn-ghost")
                b501_btn   = gr.Button("Boundary 501", elem_classes="btn-ghost")
                suite_btn  = gr.Button("Boundary Suite 5/5", elem_classes="btn-ghost")
                demo_btn   = gr.Button("Demo (Member + HBL + 1200)", elem_classes="btn-ghost")

            # Receipt section
            with gr.Group():
                gr.HTML("<div class='card'><div class='card-title'>🧾 Download Receipt</div><p>Generate first, then download (HTML → Print → Save as PDF).</p>")
                receipt_file = gr.File(label="Receipt File", interactive=False)
                gr.HTML("</div>")

            details_md = gr.Markdown(visible=False)

        with gr.Column(scale=6):
            result_html = gr.HTML(value="<div class='card slide-up'><div class='card-title'>🧾 Final Bill Summary</div><p>Fill inputs and click <b>Submit</b>.</p></div>")

    gr.Markdown("### 🧪 Calculation History (current session)")
    history_df = gr.Dataframe(headers=["Time","Amount","Customer","Payment","Discount %","Final (Rs)"], interactive=False, wrap=True)

    # --------- Events ---------
    def on_submit(a, c, p, hist_df):
        html, det, row, receipt_path = compute_bill(a, c, p)
        if isinstance(hist_df, pd.DataFrame) and not hist_df.empty:
            out_df = pd.concat([hist_df, row], ignore_index=True)
        else:
            out_df = row
        return html, gr.update(value=det, visible=True), out_df, out_df, receipt_path

    def on_clear():
        return (
            "<div class='card slide-up'><div class='card-title'>🧾 Final Bill Summary</div><p>Fill inputs and click <b>Submit</b>.</p></div>",
            gr.update(visible=False),
            pd.DataFrame(), pd.DataFrame(), None, "600", "Guest", "Other"
        )

    submit_btn.click(on_submit, [amount, cust, pay, history_state], [result_html, details_md, history_df, history_state, receipt_file])
    clear_btn.click(on_clear, outputs=[result_html, details_md, history_df, history_state, receipt_file, amount, cust, pay])

    b500_btn.click(lambda: ("500","Guest","Other"), outputs=[amount, cust, pay])
    b501_btn.click(lambda: ("501","Guest","Other"), outputs=[amount, cust, pay])
    demo_btn.click(lambda: ("1200","Member","HBL"), outputs=[amount, cust, pay])

    # Boundary 5/5 suite — uses current customer/payment; shows multi-cards & appends 5 rows
    suite_btn.click(boundary_suite, [cust, pay, history_state], [result_html, details_md, history_df, history_state, receipt_file])

app.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>